# Libraries

In [ ]:
import os
import sys
import pickle
import datetime, time
import random
from importlib import reload
from math import sqrt
import random

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

module_paths = {os.pardir}  # package directory
sys.path.extend(module_paths.difference(sys.path))


from nytf import utils; reload(utils)
#from nytf.main_pipeline import main_estimator

# Loading dataframes

In [ ]:
%%time
train = utils.load_dataframe('train')#, cloud=False)
train = pd.concat((train, utils.BasicTemporalFeatures().transform(train)), axis=1)

#with open(os.path.join(utils.PROCESSING_DIRECTORY, 'train_basic_features.pkl'), 'rb') as f:
#    train = pickle.load(f)

test = utils.load_dataframe('test')#, cloud=False)
test = pd.concat((test, utils.BasicTemporalFeatures().transform(test)), axis=1)

In [ ]:
train.memory_usage().sum()/10**9  # In Go

# RandomForest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
%%time
N = 200000
ind = sorted(random.sample(range(train.shape[0]), N))
X = train.iloc[ind][['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'passenger_count',
           'timestamp', 'week_progress_cos', 'week_progress_sin', 'month_progress_cos', 'month_progress_sin',
           'year_progress_cos', 'year_progress_sin', 'day_progress_cos', 'day_progress_sin']].values
y = train.iloc[ind]['fare_amount'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.5)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
regr = RandomForestRegressor()

In [ ]:
%%time
regr.fit(X_train, y_train)

In [ ]:
%%time
y_train_est = regr.predict(X_train).clip(0, 100)
print('Train error :', sqrt(((y_train_est - y_train) ** 2).mean()))
y_test_est = regr.predict(X_test).clip(0, 100)
print('Test error :', sqrt(((y_test_est - y_test) ** 2).mean()))

# NeuralNetwork 1

In [ ]:
import tensorflow as tf
from tensorflow.python.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
%%time
X = train[['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'passenger_count',
           'timestamp', 'week_progress_cos', 'week_progress_sin', 'month_progress_cos', 'month_progress_sin',
           'year_progress_cos', 'year_progress_sin', 'day_progress_cos', 'day_progress_sin']].values
y = train['fare_amount'].values

scaler = StandardScaler()
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.05)
X = scaler.fit_transform(X)
#X_train = scaler.fit_transform(X_train)
#X_test = scaler.transform(X_test)

In [ ]:
def rmse(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(y_true, y_pred))))

In [ ]:
model = tf.keras.Sequential()
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation=lambda tensor: tf.keras.activations.relu(tensor, max_value=100)))


In [ ]:
model.compile(optimizer=tf.train.AdamOptimizer(0.01), loss='mse', metrics=[rmse])
#model.compile(optimizer=tf.keras.optimizers.SGD(0.01), loss='mse', metrics=[rmse])

In [ ]:
model.fit(X, y, epochs=100, batch_size=2**20)

In [ ]:
#model.save_weights('test.h5')

In [ ]:
model.summary()

In [ ]:
X_test = test[['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'passenger_count',
           'timestamp', 'week_progress_cos', 'week_progress_sin', 'month_progress_cos', 'month_progress_sin',
           'year_progress_cos', 'year_progress_sin', 'day_progress_cos', 'day_progress_sin']].values

X_test = scaler.transform(X_test)
y_test_pred = model.predict(X_test)

plt.hist(y_test_pred, bins=200)
pass

In [ ]:
test['fare_amount'] = y_test_pred

In [ ]:
#test.to_csv('first_neural_network.csv', index=False, columns=['key', 'fare_amount'])

# NeuralNetwork 2

In [ ]:
import tensorflow as tf
from tensorflow.python.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
%%time
X = train[['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'passenger_count',
           'timestamp', 'week_progress_cos', 'week_progress_sin', 'month_progress_cos', 'month_progress_sin',
           'year_progress_cos', 'year_progress_sin', 'day_progress_cos', 'day_progress_sin']].astype('float32')
y = train['fare_amount'].astype('float32').values

del train

latitude_scaler, longitude_scaler, timestamp_scaler = StandardScaler(), StandardScaler(), StandardScaler()
X['pickup_longitude'], X['dropoff_longitude'] = longitude_scaler.fit_transform(X['pickup_longitude'].values.reshape(-1, 1)), longitude_scaler.transform(X['dropoff_longitude'].values.reshape(-1, 1))
X['pickup_latitude'], X['dropoff_latitude'] = latitude_scaler.fit_transform(X['pickup_latitude'].values.reshape(-1, 1)), latitude_scaler.transform(X['dropoff_latitude'].values.reshape(-1, 1))
X['timestamp'] = timestamp_scaler.fit_transform(X['timestamp'].values.reshape(-1, 1))

X_coordinate_pickup = X[['pickup_longitude', 'pickup_latitude']].values
X_coordinate_dropoff = X[['dropoff_longitude', 'dropoff_latitude']].values
X_other = X.drop(columns=['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude']).values

del X

In [ ]:
%%time
X_coordinate_pickup_train, X_coordinate_pickup_test, X_coordinate_dropoff_train, X_coordinate_dropoff_test, X_other_train, X_other_test, y_train, y_test = train_test_split(X_coordinate_pickup, X_coordinate_dropoff, X_other, y, test_size=.01)

In [ ]:
other_dim = X_other_train.shape[1]
one_coordinate_sequence = [8, 4]
two_coordinates_sequence = [16, 32, 16]
end_sequence = [64, 64, 32, 16]

one_coordinate_model = tf.keras.Sequential()
for i, dim in enumerate(one_coordinate_sequence):
  if i==0:
    one_coordinate_model.add(layers.Dense(dim, input_dim=2, activation='relu'))
  else:
    one_coordinate_model.add(layers.Dense(dim, activation='relu'))

two_coordinates_model = tf.keras.Sequential()
for i, dim in enumerate(two_coordinates_sequence):
  if i==0:
    two_coordinates_model.add(layers.Dense(dim, input_dim=2*one_coordinate_sequence[-1], activation='relu'))
  else:
    two_coordinates_model.add(layers.Dense(dim, activation='relu'))

end_model = tf.keras.Sequential()
for i, dim in enumerate(end_sequence):
  if i==0:
    end_model.add(layers.Dense(dim, input_dim=other_dim+4+two_coordinates_sequence[-1], activation='relu'))
  else:
    end_model.add(layers.Dense(dim, activation='relu'))

final_activation = lambda tensor: tf.keras.activations.relu(tensor, max_value=100)

pickup_coordinate_input = layers.Input(shape=(2,), name='pickup_coordinate_input')
dropoff_coordinate_input = layers.Input(shape=(2,), name='dropoff_coordinate_input')
other_input = layers.Input(shape=(other_dim,), name='other_input')

pickup_coordinate_vect = one_coordinate_model(pickup_coordinate_input)
dropoff_coordinate_vect = one_coordinate_model(dropoff_coordinate_input)
two_coordinates_vect = two_coordinates_model(layers.Concatenate()([pickup_coordinate_vect, dropoff_coordinate_vect]))
end_vect = end_model(layers.Concatenate()([two_coordinates_vect, pickup_coordinate_input,
                                           dropoff_coordinate_input, other_input]))

coordinates_output = layers.Dense(1, activation=final_activation, name='coordinates_output')(two_coordinates_vect)
total_output = layers.Dense(1, activation=final_activation, name='total_output')(end_vect)

model = tf.keras.Model(inputs=[pickup_coordinate_input, dropoff_coordinate_input, other_input],
                       outputs=[total_output, coordinates_output])

model.summary()

In [ ]:
def rmse(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(y_true, y_pred))))

model.compile(optimizer=tf.train.AdamOptimizer(0.01),
              #optimizer=tf.keras.optimizers.SGD(0.01),
              loss='mse', metrics=[rmse], loss_weights={'total_output': 1., 'coordinates_output': .2})

In [ ]:
model.fit({'pickup_coordinate_input': X_coordinate_pickup_train,
           'dropoff_coordinate_input': X_coordinate_dropoff_train,
           'other_input': X_other_train},
          {'total_output': y_train, 'coordinates_output': y_train},
          validation_data=({'pickup_coordinate_input': X_coordinate_pickup_test,
           'dropoff_coordinate_input': X_coordinate_dropoff_test,
           'other_input': X_other_test},
          {'total_output': y_test, 'coordinates_output': y_test}),
          epochs=100, batch_size=2**19)

In [ ]:
%%time
X_test2 = test[['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'passenger_count',
           'timestamp', 'week_progress_cos', 'week_progress_sin', 'month_progress_cos', 'month_progress_sin',
           'year_progress_cos', 'year_progress_sin', 'day_progress_cos', 'day_progress_sin']].astype('float32')

X_test2['pickup_longitude'], X_test2['dropoff_longitude'] = longitude_scaler.transform(X_test2['pickup_longitude'].values.reshape(-1, 1)), longitude_scaler.transform(X_test2['dropoff_longitude'].values.reshape(-1, 1))
X_test2['pickup_latitude'], X_test2['dropoff_latitude'] = latitude_scaler.transform(X_test2['pickup_latitude'].values.reshape(-1, 1)), latitude_scaler.transform(X_test2['dropoff_latitude'].values.reshape(-1, 1))
X_test2['timestamp'] = timestamp_scaler.transform(X_test2['timestamp'].values.reshape(-1, 1))

X_test2_coordinate_pickup = X_test2[['pickup_longitude', 'pickup_latitude']].values
X_test2_coordinate_dropoff = X_test2[['dropoff_longitude', 'dropoff_latitude']].values
X_test2_other = X_test2.drop(columns=['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude']).values